<a href="https://colab.research.google.com/github/Luca-wirthner/2024_Geoinformatique_I/blob/main/script_zones_tampons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import geopandas as gpd
import pyogrio as pgr
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# The file can be accessed by using its name frome the colab directory
pgr.list_layers('zone_etude.gpkg')

array([['ze_3860', 'MultiPolygon'],
       ['perimetre_evaluation', 'Polygon']], dtype=object)

In [2]:
zone_etude = gpd.read_file('zone_etude.gpkg', layer = 'ze_3860')

In [3]:
buff_500m = zone_etude.buffer(500).union_all()
buff_1km_dif = zone_etude.buffer(1000).union_all() - zone_etude.buffer(500).union_all()
buff_5km_dif = zone_etude.buffer(5000).union_all() - zone_etude.buffer(1000).union_all()

In [4]:
buffs = {'500m': buff_500m, '1km': buff_1km_dif, '5km': buff_5km_dif}
gdf_buffs = { k: gpd.GeoDataFrame(geometry=[poly], crs='EPSG:2056') for k, poly in buffs.items()}

In [7]:
pgr.list_layers('admin.gpkg')

array([['communes', 'MultiPolygon Z'],
       ['cantons', 'MultiPolygon Z']], dtype=object)

In [5]:
communes = gpd.read_file('admin.gpkg', layer = 'communes')

In [6]:
gdf_buffs_cmns = { k: gpd.overlay(gdf_buffs[k], communes, how = 'intersection') for k in gdf_buffs}

In [10]:
gdf_buffs_cmns['500m']


,id,uuid,datum_aenderung,datum_erstellung,erstellung_jahr,erstellung_monat,grund_aenderung,herkunft,herkunft_jahr,herkunft_monat,...,bezirksnummer,kantonsnummer,name,gem_flaeche,see_flaeche,icc,shn,einwohnerzahl,hist_nr,geometry
0,537,{88D27170-ED1E-45BF-952B-D6ECCDFDC6E7},2023-11-08,2008-11-24,2006,1,Verbessert,swisstopo,2024,1,...,2224.0,22.0,Yverdon-les-Bains,1503.0,148.0,CH,CH22245938,29827,15497.0,"POLYGON Z ((2540127.532 1180161.316 509.523, 2..."
1,843,{F03B2FDF-837A-4B56-93B1-332E8BBFAC37},2023-10-24,2008-11-24,2006,1,Verbessert,swisstopo,2024,1,...,2224.0,22.0,Pomy,562.0,0.0,CH,CH22245926,864,14579.0,MULTIPOLYGON Z (((2540341.983 1179921.825 508....


In [12]:
ppgr.list_layers('stat_ofs.gpkg')

array([['pop_2023', 'Point'],
       ['empl_2022', 'Point']], dtype=object)

In [7]:
population = gpd.read_file('stat_ofs.gpkg', layer = 'pop_2023')
emplois = gpd.read_file('stat_ofs.gpkg', layer = 'empl_2022')

In [8]:
def sjoin_summarize(poly_lyr, pts_lyr, pts_colname, poly_newcolname):
    """
    poly_lyr:         la couche des polygones
    pts_lyr:          la couche des points
    pts_colname:      la colonne sur laquelle il faut faire la somme
    poly_newcolname:  le nom de la colonne à créer dans la couche
                      des polygones
    """
    # Faire d'abord une jointure spatiale pour associer chaque point à un polygone
    sjoin = pts_lyr.sjoin(poly_lyr, how='inner', predicate='within')

    # Faire une opération «group by» selon l'index du polygone, et calculer la somme
    somme_par_poly = sjoin.groupby(sjoin.index_right)[pts_colname].sum()

    # Enregistrer le résultat dans la couche des polygones
    poly_lyr[poly_newcolname] = poly_lyr.index.map(somme_par_poly)

In [16]:
population

,BBTOT,E_KOORD,N_KOORD,geometry
0,17,2529800,1171500,POINT (2529800 1171500)
1,29,2529800,1171600,POINT (2529800 1171600)
2,25,2529800,1171700,POINT (2529800 1171700)
3,12,2529800,1171800,POINT (2529800 1171800)
4,11,2529800,1171900,POINT (2529800 1171900)
...,...,...,...,...
3319,9,2549700,1181000,POINT (2549700 1181000)
3320,17,2549700,1183200,POINT (2549700 1183200)
3321,18,2549700,1183300,POINT (2549700 1183300)
3322,3,2549700,1184600,POINT (2549700 1184600)


In [17]:
emplois

,B08EMPT,E_KOORD,N_KOORD,geometry
0,4,2529800,1171500,POINT (2529800 1171500)
1,4,2529800,1171600,POINT (2529800 1171600)
2,4,2529800,1171700,POINT (2529800 1171700)
3,4,2529800,1171800,POINT (2529800 1171800)
4,4,2529800,1172100,POINT (2529800 1172100)
...,...,...,...,...
2158,4,2549700,1174800,POINT (2549700 1174800)
2159,4,2549700,1180300,POINT (2549700 1180300)
2160,4,2549700,1180400,POINT (2549700 1180400)
2161,4,2549700,1181000,POINT (2549700 1181000)


In [21]:
sjoin_summarize(gdf_buffs_cmns['500m'], population, 'BBTOT', 'pop_2023_sum')
sjoin_summarize(gdf_buffs_cmns['500m'], emplois, 'B08EMPT', 'empl_2022_sum')
sjoin_summarize(gdf_buffs_cmns['1km'], population, 'BBTOT', 'pop_2023_sum')
sjoin_summarize(gdf_buffs_cmns['1km'], emplois, 'B08EMPT', 'empl_2022_sum')
sjoin_summarize(gdf_buffs_cmns['5km'], population, 'BBTOT', 'pop_2023_sum')
sjoin_summarize(gdf_buffs_cmns['5km'], emplois, 'B08EMPT', 'empl_2022_sum')

In [25]:
gdf_buffs_cmns['500m']['peri'] = '500m'
gdf_buffs_cmns['1km']['peri'] = '1km'
gdf_buffs_cmns['5km']['peri'] = '5km'

In [9]:
for k in gdf_buffs_cmns:
    sjoin_summarize(gdf_buffs_cmns[k], population, 'BBTOT', 'pop_2023_sum')
    sjoin_summarize(gdf_buffs_cmns[k], emplois, 'B08EMPT', 'empl_2022_sum')
    gdf_buffs_cmns[k]['peri'] = k

In [10]:
gdf_buffs_cmns['500m']

,id,uuid,datum_aenderung,datum_erstellung,erstellung_jahr,erstellung_monat,grund_aenderung,herkunft,herkunft_jahr,herkunft_monat,...,gem_flaeche,see_flaeche,icc,shn,einwohnerzahl,hist_nr,geometry,pop_2023_sum,empl_2022_sum,peri
0,537,{88D27170-ED1E-45BF-952B-D6ECCDFDC6E7},2023-11-08,2008-11-24,2006,1,Verbessert,swisstopo,2024,1,...,1503.0,148.0,CH,CH22245938,29827,15497.0,"POLYGON Z ((2540127.532 1180161.316 509.523, 2...",3049,5097,500m
1,843,{F03B2FDF-837A-4B56-93B1-332E8BBFAC37},2023-10-24,2008-11-24,2006,1,Verbessert,swisstopo,2024,1,...,562.0,0.0,CH,CH22245926,864,14579.0,MULTIPOLYGON Z (((2540341.983 1179921.825 508....,68,88,500m


In [11]:
data_500m = gdf_buffs_cmns['500m'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]
data_1km = gdf_buffs_cmns['1km'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]
data_5km = gdf_buffs_cmns['5km'][['bfs_nummer', 'name', 'peri', 'pop_2023_sum', 'empl_2022_sum']]

In [12]:
tabl = pd.concat([data_500m, data_1km, data_5km], ignore_index=True)
tabl

,bfs_nummer,name,peri,pop_2023_sum,empl_2022_sum
0,5938,Yverdon-les-Bains,500m,3049.0,5097.0
1,5926,Pomy,500m,68.0,88.0
2,5938,Yverdon-les-Bains,1km,4973.0,1330.0
3,5926,Pomy,1km,46.0,30.0
4,5934,Valeyres-sous-Ursins,1km,NaN,NaN
5,5930,Suscévaz,5km,36.0,4.0
6,5939,Yvonand,5km,9.0,4.0
7,5520,Essertines-sur-Yverdon,5km,450.0,92.0
8,5932,Ursins,5km,240.0,41.0
9,5561,Grandson,5km,2586.0,1042.0


In [30]:
personnes_par_peri_et_cmne = tabl.replace(np.nan,0)

In [31]:
personnes_par_peri_et_cmne

,bfs_nummer,name,peri,pop_2023_sum,empl_2022_sum,Njsem,Nnsem,Njwe,Nnwe
0,5938,Yverdon-les-Bains,500m,3049.0,5097.0,4992.3,2998.95,2084.25,3049.0
1,5926,Pomy,500m,68.0,88.0,90.8,65.60,45.20,68.0
2,5938,Yverdon-les-Bains,1km,4973.0,1330.0,2555.9,4542.20,3050.30,4973.0
3,5926,Pomy,1km,46.0,30.0,37.8,42.90,29.10,46.0
4,5934,Valeyres-sous-Ursins,1km,0.0,0.0,0.0,0.00,0.00,0.0
5,5930,Suscévaz,5km,36.0,4.0,14.0,32.60,21.80,36.0
6,5939,Yvonand,5km,9.0,4.0,5.9,8.30,5.60,9.0
7,5520,Essertines-sur-Yverdon,5km,450.0,92.0,208.6,409.60,274.60,450.0
8,5932,Ursins,5km,240.0,41.0,104.8,218.05,146.05,240.0
9,5561,Grandson,5km,2586.0,1042.0,1609.4,2379.50,1603.70,2586.0


In [32]:
'''Njsem: Nombre de personnes présentes pendant un jour de semaine
   Nnsem: Nombre de personnes présentes pendant une nuit de semaine
   Njwe: Nombre de personnes présentes pendant un jour de weekend
   Nnwe: Nombre de personnes présentes pendant une nuit de weekend'''
personnes_par_peri_et_cmne['Njsem'] = ((0.3*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.8*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Nnsem'] = ((0.9*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.05*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Njwe'] = ((0.6*personnes_par_peri_et_cmne['pop_2023_sum']) + (0.05*personnes_par_peri_et_cmne['empl_2022_sum']))
personnes_par_peri_et_cmne['Nnwe'] = personnes_par_peri_et_cmne['pop_2023_sum']

In [33]:
personnes_par_peri_et_cmne

,bfs_nummer,name,peri,pop_2023_sum,empl_2022_sum,Njsem,Nnsem,Njwe,Nnwe
0,5938,Yverdon-les-Bains,500m,3049.0,5097.0,4992.3,2998.95,2084.25,3049.0
1,5926,Pomy,500m,68.0,88.0,90.8,65.60,45.20,68.0
2,5938,Yverdon-les-Bains,1km,4973.0,1330.0,2555.9,4542.20,3050.30,4973.0
3,5926,Pomy,1km,46.0,30.0,37.8,42.90,29.10,46.0
4,5934,Valeyres-sous-Ursins,1km,0.0,0.0,0.0,0.00,0.00,0.0
5,5930,Suscévaz,5km,36.0,4.0,14.0,32.60,21.80,36.0
6,5939,Yvonand,5km,9.0,4.0,5.9,8.30,5.60,9.0
7,5520,Essertines-sur-Yverdon,5km,450.0,92.0,208.6,409.60,274.60,450.0
8,5932,Ursins,5km,240.0,41.0,104.8,218.05,146.05,240.0
9,5561,Grandson,5km,2586.0,1042.0,1609.4,2379.50,1603.70,2586.0


In [34]:
personnes_par_peri = personnes_par_peri_et_cmne[['peri','Njsem','Nnsem','Njwe','Nnwe']].groupby('peri', as_index = False).sum()

In [39]:
personnes_par_peri

,peri,Njsem,Nnsem,Njwe,Nnwe
0,1km,2593.7,4585.10,3079.40,5019.0
1,500m,5083.1,3064.55,2129.45,3117.0
2,5km,25488.9,29689.95,20125.05,31883.0


In [35]:
personnes_par_cmne = personnes_par_peri_et_cmne[['bfs_nummer','name','Njsem','Nnsem','Njwe','Nnwe']].groupby(['bfs_nummer','name'], as_index = False).sum()

In [40]:
personnes_par_cmne

,bfs_nummer,name,Njsem,Nnsem,Njwe,Nnwe
0,5520,Essertines-sur-Yverdon,208.6,409.60,274.60,450.0
1,5559,Giez,5.7,3.00,2.10,3.0
2,5561,Grandson,1609.4,2379.50,1603.70,2586.0
3,5902,Belmont-sur-Yverdon,211.4,399.20,267.80,438.0
4,5903,Bioley-Magnoux,0.0,0.00,0.00,0.0
5,5904,Chamblon,388.4,521.30,352.10,564.0
6,5905,Champvent,0.0,0.00,0.00,0.0
7,5909,Cheseaux-Noréaz,456.3,642.75,433.65,697.0
8,5910,Cronay,206.3,383.90,257.60,421.0
9,5911,Cuarny,139.2,234.30,157.50,256.0


In [37]:
personnes_par_peri_et_cmne.to_csv('personnes_par_peri_et_cmne.csv', index =False)
personnes_par_peri.to_csv('personnes_par_peri.csv', index = False)
personnes_par_cmne.to_csv('personnes_par_cmne.csv', index = False)

In [38]:
from google.colab import files
files.download('personnes_par_peri_et_cmne.csv')
files.download('personnes_par_peri.csv')
files.download('personnes_par_cmne.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>